In [1]:
#hide
# default_exp test
from nbdev.showdoc import *

# Test `kicad-helpers` scripts on a demo repository

* [ ] git clone test repository (e.g., https://github.com/sci-bots/dropbot-40-channel-HV-switching-board.kicad) into temp directory
* [ ] run kh_update
* [ ] run tests
* [ ] test the .github/workflows?

In [2]:
#export
import os
import subprocess

from fastcore.script import *
from nbdev.test import *
from nbdev.test import nbglob, num_cpus, parallel, _test_one, Path
from kicad_helpers import get_git_root, setup_test_repo, get_project_name, get_schematic_path, get_bom_path
from kicad_helpers.core import _set_root

In [3]:
#export
@call_parse
def test_notebooks(fname:Param("A notebook name or glob to convert", str)=None,
                   flags:Param("Space separated list of flags", str)=None,
                   n_workers:Param("Number of workers to use", int)=None,
                   verbose:Param("Print errors along the way", bool_arg)=True,
                   timing:Param("Timing each notebook to see the ones are slow", bool)=False,
                   pause:Param("Pause time (in secs) between notebooks to avoid race conditions", float)=0.5,
                   root:Param("project root directory", str)="."):
    "Test in parallel the notebooks matching `fname`, passing along `flags`"
    root = _set_root(root)
    if flags is not None: flags = flags.split(' ')
    if fname is None:
        fname = os.path.join(root, "tests", "*.ipynb")
    files = nbglob(fname, recursive=False)
    files = [Path(f).absolute() for f in sorted(files)]
    assert len(files) > 0, "No files to test found."
    if n_workers is None: n_workers = 0 if len(files)==1 else min(num_cpus(), 8)
    # make sure we are inside the tests folder
    os.chdir(os.path.join(root, "tests"))
    results = parallel(_test_one, files, flags=flags, verbose=verbose, n_workers=n_workers, pause=pause)
    passed,times = [r[0] for r in results],[r[1] for r in results]
    if all(passed): print("All tests are passing!")
    else:
        msg = "The following notebooks failed:\n"
        raise Exception(msg + '\n'.join([f.name for p,f in zip(passed,files) if not p]))
    if timing:
        for i,t in sorted(enumerate(times), key=lambda o:o[1], reverse=True):
            print(f"Notebook {files[i].name} took {int(t)} seconds")

In [4]:
#hide
root = os.path.join(get_git_root("."), "_temp")
setup_test_repo(root)

Cloning into '/mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp'...
Updating files: 100% (60/60), done.


In [5]:
#hide_input
print(subprocess.check_output(f"kh_update --overwrite --root { root }", shell=True).decode("utf-8"))

In [6]:
print(subprocess.check_output(f"kh_test --root { root }", shell=True).decode("utf-8"))

root=/mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp
fname=/mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/tests/*.ipynb
files=[Path('/mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/tests/Tests.ipynb')]
testing /mnt/c/Users/ryan/OneDrive/dev/python/kicad-helpers/_temp/tests/Tests.ipynb
All tests are passing!

